In [18]:
from PIL import Image, ImageDraw
import os
import pandas as pd
import glob
import numpy as np
import shutil
import  xml.dom.minidom
from tqdm.auto import tqdm

In [23]:
TEST_IMAGES_INPUT_DIR = r"C:\project\Computer_Vision\term_paper_2023\datasets\test_Anti-UAV\test\img"
TEST_LABELS_INPUT_DIR = r"C:\project\Computer_Vision\term_paper_2023\datasets\test_Anti-UAV\test\xml"

VALID_IMAGES_INPUT_DIR = r"C:\project\Computer_Vision\term_paper_2023\datasets\val_Anti-UAV\val\img"
VALID_LABELS_INPUT_DIR = r"C:\project\Computer_Vision\term_paper_2023\datasets\val_Anti-UAV\val\xml"

TRAIN_IMAGES_INPUT_DIR = r"C:\project\Computer_Vision\term_paper_2023\datasets\antiuav_train\img"
TRAIN_LABELS_INPUT_DIR = r"C:\project\Computer_Vision\term_paper_2023\datasets\antiuav_train\xml"

YOLO_DIR = r"C:\project\Computer_Vision\term_paper_2023\datasets\Anti_UAV_yolo"

TEST_IMAGES_YOLO_DIR = os.path.join(YOLO_DIR, "test", "images")
TEST_LABELS_YOLO_DIR = os.path.join(YOLO_DIR, "test", "labels")

VALID_IMAGES_YOLO_DIR = os.path.join(YOLO_DIR, "valid", "images")
VALID_LABELS_YOLO_DIR = os.path.join(YOLO_DIR, "valid", "labels")

TRAIN_IMAGES_YOLO_DIR = os.path.join(YOLO_DIR, "train", "images")
TRAIN_LABELS_YOLO_DIR = os.path.join(YOLO_DIR, "train", "labels")

if os.path.exists(YOLO_DIR):
    shutil.rmtree(YOLO_DIR)

os.makedirs(os.path.join(TEST_IMAGES_YOLO_DIR))
os.makedirs(os.path.join(TEST_LABELS_YOLO_DIR))
os.makedirs(os.path.join(VALID_IMAGES_YOLO_DIR))
os.makedirs(os.path.join(VALID_LABELS_YOLO_DIR))
os.makedirs(os.path.join(TRAIN_IMAGES_YOLO_DIR))
os.makedirs(os.path.join(TRAIN_LABELS_YOLO_DIR))

def parse_xml(path_xml):
    dom = xml.dom.minidom.parse(path_xml)
    coords_drons = []
    for i in range(len(dom.getElementsByTagName('xmin'))):
        xmin = int(dom.getElementsByTagName('xmin')[i].childNodes[0].data)
        ymin = int(dom.getElementsByTagName('ymin')[i].childNodes[0].data)
        xmax = int(dom.getElementsByTagName('xmax')[i].childNodes[0].data)
        ymax = int(dom.getElementsByTagName('ymax')[i].childNodes[0].data)
        coords_drons.append((xmin, ymin, xmax, ymax))

    width = int(dom.getElementsByTagName('width')[0].childNodes[0].data)
    height = int(dom.getElementsByTagName('height')[0].childNodes[0].data)
    return coords_drons , width, height

def get_df(images_path, labels_path):
    df = pd.DataFrame (glob.glob( os.path.join(images_path,'*.jpg')), columns=['path_image'])
    df["id"] = df['path_image'].apply(lambda x : x.split('\\')[-1].split('.')[0])
    df["path_label"] = df['id'].apply(lambda x : os.path.join(labels_path,f"{x}.xml"))
    return df

def check_yolo_label(image, x_center_yolo, y_center_yolo, width_drone_yolo, height_drone_yolo):

    width, height = image.size
    x0 = (x_center_yolo - width_drone_yolo/2)*width
    y0 = (y_center_yolo - height_drone_yolo/2)*height

    x1 = (x_center_yolo + width_drone_yolo/2)*width
    y1 = (y_center_yolo - height_drone_yolo/2)*height

    x2 = (x_center_yolo + width_drone_yolo/2)*width
    y2 = (y_center_yolo + height_drone_yolo/2)*height

    x3 = (x_center_yolo - width_drone_yolo/2)*width
    y3 = (y_center_yolo + height_drone_yolo/2)*height

    draw_image = ImageDraw.Draw(image)
    draw_image.line(((x0, y0), (x1, y1), (x2, y2), (x3, y3), (x0, y0)), fill=(255, 0, 0))
    image.show()

In [24]:
def dataset2yolo(df, images_path, labels_path):
    for i, row in tqdm(df.iterrows()):
        coords_drons , width_image, height_image = parse_xml(row['path_label'])
        str_label_yolo = ""
        for xmin, ymin, xmax, ymax in coords_drons:

            width_drone = (xmax - xmin)
            height_drone = (ymax - ymin)
            width_drone_yolo = width_drone / width_image
            height_drone_yolo = height_drone / height_image

            x_center_yolo = (xmin + width_drone / 2) / width_image
            y_center_yolo = (ymin + height_drone / 2) / height_image

            str_label_yolo += f"0 {x_center_yolo} {y_center_yolo} {width_drone_yolo} {height_drone_yolo} \n"

            #CHECK LABEL
            # if i<5:
            #     check_yolo_label(Image.open(row['path_image']), x_center_yolo, y_center_yolo, width_drone_yolo, height_drone_yolo )

        path_label_output = os.path.join(labels_path, row['id'] + '.txt')

        with open(path_label_output, 'w') as f:
            f.write(str_label_yolo)
        image_save_path = os.path.join(images_path, f"{row['id']}.jpg")
        Image.open(row['path_image']).resize((512,512)).save(image_save_path)
        # shutil.copy(row['path_image'], os.path.join(images_path, f"{row['id']}.jpg"))

In [25]:
df_test = get_df(TEST_IMAGES_INPUT_DIR, TEST_LABELS_INPUT_DIR)
df_valid = get_df(VALID_IMAGES_INPUT_DIR, VALID_LABELS_INPUT_DIR)
df_train = get_df(TRAIN_IMAGES_INPUT_DIR, TRAIN_LABELS_INPUT_DIR)

In [26]:
dataset2yolo(df_test,TEST_IMAGES_YOLO_DIR, TEST_LABELS_YOLO_DIR)
dataset2yolo(df_valid,VALID_IMAGES_YOLO_DIR, VALID_LABELS_YOLO_DIR)
dataset2yolo(df_train,TRAIN_IMAGES_YOLO_DIR, TRAIN_LABELS_YOLO_DIR)
shutil.make_archive(YOLO_DIR, 'zip', YOLO_DIR)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

'C:\\project\\Computer_Vision\\term_paper_2023\\datasets\\Anti_UAV_yolo.zip'